[texto do link](https://)# **Data Science na Copa do Mundo Qatar 2022 ⚽**

In [1]:
import pandas as pd
import numpy as np
import random
from scipy.stats import poisson

selecoes = pd.read_excel('DadosCopaDoMundoQatar2022.xlsx', sheet_name ='selecoes', index_col = 0)
jogos = pd.read_excel('DadosCopaDoMundoQatar2022.xlsx', sheet_name ='jogos')


In [2]:
selecoes.head()

,Grupo,GrupoNumero,NomeEmIngles,PosiçãoRankingFIFA,ValorDeMercado,PontosRankingFIFA,Confederação,Copas,JogadorDestaque,FotoJogadorDestaque,LinkBandeiraPequena,LinkBandeiraGrande
Seleção,,,,,,,,,,,,
Catar,A,A1,Qatar,48,18.33,1442,AFC,0,Almoez Ali,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Equador,A,A2,Ecuador,44,125.80,1464,UEFA,0,Pervis Estupiñán,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Senegal,A,A3,Senegal,18,286.50,1585,CAF,0,Sadio Mané,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Holanda,A,A4,Netherlands,8,455.75,1679,UEFA,0,Virgil Van Dijk,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Inglaterra,B,B1,England,5,1360.00,1737,UEFA,1,Harry Kane,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...


In [3]:
jogos.head()

,data,hora,grupo,seleção1,seleção2,estádio
0,2022-11-21,07:00:00,A,Senegal,Holanda,AL THUMAMA
1,2022-11-21,13:00:00,A,Catar,Equador,AL KHOR
2,2022-11-25,10:00:00,A,Catar,Senegal,AL THUMAMA
3,2022-11-25,13:00:00,A,Holanda,Equador,INTER. KHALIFA
4,2022-11-29,12:00:00,A,Holanda,Catar,AL KHOR


In [4]:
fifa = selecoes['PontosRankingFIFA']
fifa

Seleção
Catar             1442
Equador           1464
Senegal           1585
Holanda           1679
Inglaterra        1737
Irã               1559
Estados Unidos    1635
País de Gales     1582
Argentina         1771
Arábia Saudita    1436
México            1650
Polônia           1546
França            1765
Dinamarca         1665
Tunísia           1508
Austrália         1484
Espanha           1717
Alemanha          1659
Japão             1555
Costa Rica        1500
Bélgica           1822
Canadá            1474
Marrocos          1558
Croácia           1632
Brasil            1838
Sérvia            1550
Suíça             1621
Camarões          1485
Portugal          1679
Gana              1393
Uruguai           1641
Coreia do Sul     1526
Name: PontosRankingFIFA, dtype: int64

In [5]:
a, b = min(fifa), max(fifa) 
fa, fb = 0.15, 1 
b1 = (fb - fa)/(b-a) 
b0 = fb - b*b1
forca = b0 + b1*fifa 

forca.sort_values(ascending = False)

Seleção
Brasil            1.000000
Bélgica           0.969438
Argentina         0.872022
França            0.860562
Inglaterra        0.807079
Espanha           0.768876
Holanda           0.696292
Portugal          0.696292
Dinamarca         0.669551
Alemanha          0.658090
México            0.640899
Uruguai           0.623708
Estados Unidos    0.612247
Croácia           0.606517
Suíça             0.585506
Senegal           0.516742
País de Gales     0.511011
Irã               0.467079
Marrocos          0.465169
Japão             0.459438
Sérvia            0.449888
Polônia           0.442247
Coreia do Sul     0.404045
Tunísia           0.369663
Costa Rica        0.354382
Camarões          0.325730
Austrália         0.323820
Canadá            0.304719
Equador           0.285618
Catar             0.243596
Arábia Saudita    0.232135
Gana              0.150000
Name: PontosRankingFIFA, dtype: float64

In [6]:
def Resultado(gols1, gols2):
    if gols1 > gols2:
        res = 'V'
    if gols1 < gols2:
        res = 'D' 
    if gols1 == gols2:
        res = 'E'       
    return res

def MediasPoisson(selecao1, selecao2):
    forca1 = forca[selecao1]
    forca2 = forca[selecao2] 
    mgols = 2.75
    l1 = mgols*forca1/(forca1 + forca2)
    l2 = mgols*forca2/(forca1 + forca2)
    return [l1, l2]
    
def Distribuicao(media):
    probs = []
    for i in range(7):
        probs.append(poisson.pmf(i,media))
    probs.append(1-sum(probs))
    return pd.Series(probs, index = ['0', '1', '2', '3', '4', '5', '6', '7+'])


def Pontos(gols1, gols2):
    rst = Resultado(gols1, gols2)
    if rst == 'V':
        pontos1, pontos2 = 3, 0
    if rst == 'E':
        pontos1, pontos2 = 1, 1
    if rst == 'D':
        pontos1, pontos2 = 0, 3
    return pontos1, pontos2, rst


def Jogo(selecao1, selecao2):
    l1, l2 = MediasPoisson(selecao1, selecao2)
    gols1 = int(np.random.poisson(lam = l1, size = 1))
    gols2 = int(np.random.poisson(lam = l2, size = 1))
    saldo1 = gols1 - gols2
    saldo2 = -saldo1
    pontos1, pontos2, result = Pontos(gols1, gols2)
    placar = '{}x{}'.format(gols1, gols2)
    return [gols1, gols2, saldo1, saldo2, pontos1, pontos2, result, placar]


def ProbabilidadesPartida(selecao1, selecao2):
    l1, l2 = MediasPoisson(selecao1, selecao2)
    d1, d2 = Distribuicao(l1), Distribuicao(l2)  
    matriz = np.outer(d1, d2)    #   Monta a matriz de probabilidades

    vitoria = np.tril(matriz).sum()-np.trace(matriz)    #Soma a triangulo inferior
    derrota = np.triu(matriz).sum()-np.trace(matriz)    #Soma a triangulo superior
    empate = 1 - (vitoria + derrota)
    probs = np.around([vitoria, empate , derrota], 3)
    probsp = [f'{100*i:.1f}%' for i in probs]

    nomes = ['0', '1', '2', '3', '4', '5', '6', '7+']
    matriz = pd.DataFrame(matriz, columns = nomes, index = nomes)
    matriz.index = pd.MultiIndex.from_product([[selecao1], matriz.index])
    matriz.columns = pd.MultiIndex.from_product([[selecao2], matriz.columns]) 

    output = {'seleção1': selecao1, 'seleção2': selecao2, 
             'f1': forca[selecao1], 'f2': forca[selecao2], 
             'media1': l1, 'media2': l2, 
             'probabilidades': probsp, 'matriz': matriz}
    return output


In [7]:
Jogo('Brasil', 'França')

[3, 2, 1, -1, 3, 0, 'V', '3x2']

In [8]:
ProbabilidadesPartida('Brasil', 'França')

{'seleção1': 'Brasil',
 'seleção2': 'França',
 'f1': 1.0,
 'f2': 0.8605617977528088,
 'media1': 1.478048191315901,
 'media2': 1.2719518086840993,
 'probabilidades': ['42.0%', '25.4%', '32.6%'],
 'matriz':              França                                                    \
                   0         1         2         3         4         5   
 Brasil 0   0.063928  0.081313  0.051713  0.021926  0.006972  0.001774   
        1   0.094488  0.120185  0.076435  0.032407  0.010305  0.002622   
        2   0.069829  0.088819  0.056487  0.023950  0.007616  0.001937   
        3   0.034404  0.043760  0.027830  0.011800  0.003752  0.000955   
        4   0.012713  0.016170  0.010284  0.004360  0.001386  0.000353   
        5   0.003758  0.004780  0.003040  0.001289  0.000410  0.000104   
        6   0.000926  0.001177  0.000749  0.000318  0.000101  0.000026   
        7+  0.000239  0.000303  0.000193  0.000082  0.000026  0.000007   
 
                                    
                 

---
## Tarefa 1: **Prever os Jogos da Copa!**


In [10]:
jogos['Vitória'] = None
jogos['Empate'] = None
jogos['Derrota'] = None

for i in range(jogos.shape[0]):
    selecao1 = jogos['seleção1'][i]
    selecao2 = jogos['seleção2'][i]
    v, e, d = ProbabilidadesPartida(selecao1, selecao2)['probabilidades']
    jogos.at[i,'Vitória'] = v
    jogos.at[i,'Empate'] = e
    jogos.at[i,'Derrota'] = d

jogos.to_excel('outputEstimativasJogosCopa.xlsx')
jogos

,data,hora,grupo,seleção1,seleção2,estádio,Vitória,Empate,Derrota
0,2022-11-21,07:00:00,A,Senegal,Holanda,AL THUMAMA,28.3%,24.9%,46.8%
1,2022-11-21,13:00:00,A,Catar,Equador,AL KHOR,32.3%,25.4%,42.3%
2,2022-11-25,10:00:00,A,Catar,Senegal,AL THUMAMA,17.2%,22.1%,60.6%
3,2022-11-25,13:00:00,A,Holanda,Equador,INTER. KHALIFA,64.4%,21.0%,14.6%
4,2022-11-29,12:00:00,A,Holanda,Catar,AL KHOR,68.4%,19.6%,12.0%
5,2022-11-29,12:00:00,A,Equador,Senegal,INTER. KHALIFA,20.7%,23.3%,56.0%
6,2022-11-21,10:00:00,B,Inglaterra,Irã,INTER. KHALIFA,54.6%,23.6%,21.8%
7,2022-11-21,16:00:00,B,Estados Unidos,País de Gales,AL RAYYAN,43.0%,25.3%,31.7%
8,2022-11-25,07:00:00,B,País de Gales,Irã,AL RAYYAN,40.1%,25.5%,34.4%
9,2022-11-25,16:00:00,B,Inglaterra,Estados Unidos,AL KHOR,46.1%,25.0%,28.9%


## Tarefa 2: **Playground da Copa do Mundo!**

Permita que qualquer pessoa possa checar facilmente as previsões de probabilidade entre quaisquer duas seleções que estão na Copa.

In [12]:
def Jogos_grupo(dados,grupo):

  times = list(dados[dados['Grupo'] == grupo].index)
  time1,time2,time3,time4 = times 

  jogo1 = Jogo(time1,time2)
  jogo2 = Jogo(time3,time4)
  jogo3 = Jogo(time1,time3)
  jogo4 = Jogo(time4,time2)
  jogo5 = Jogo(time1,time4)
  jogo6 = Jogo(time3,time2)

  pt1,pt2,pt3,pt4 = 0,0,0,0
  gp1,gp2,gp3,gp4 = 0,0,0,0
  sg1,sg2,sg3,sg4 = 0,0,0,0

  gp1,gp2 = gp1 + jogo1[0] + jogo3[0] + jogo5[0], gp2 + jogo1[1] + jogo4[1] + jogo6[1]
  sg1, sg2 = sg1 + jogo1[2] + jogo3[2] + jogo5[2], sg2 + jogo1[3] + jogo4[3] + jogo6[3] 
  pt1, pt2 = pt1 + jogo1[4] + jogo3[4] + jogo5[4], pt2 + jogo1[5] + jogo4[5] + jogo6[5]
  gp3,gp4 = gp3 + jogo2[0] + jogo3[1] + jogo6[0], gp4 + jogo2[1] + jogo4[0] + jogo5[1]
  sg3, sg4 = sg3 + jogo2[2] + jogo3[3] + jogo6[2], sg4 + jogo2[3] + jogo4[2] + jogo5[3] 
  pt3, pt4 = pt3 + jogo2[4] + jogo3[5] + jogo6[4], pt4 + jogo2[5] + jogo4[4] + jogo5[5]


  tab = pd.DataFrame([[pt1,pt2,pt3,pt4],[sg1,sg2,sg3,sg4],[gp1,gp2,gp3,gp4]],columns=times,index = ['Pontos','Saldo de Gols','Gols Pró']).T
  tab.sort_values(by=['Pontos','Saldo de Gols','Gols Pró'],ascending=False,inplace=True)
  tab['Posição'] = [1,2,3,4]

  partidas = [time1 + ' x ' + time2,
              time3 + ' x ' + time4,
              time1 + ' x ' + time3,
              time4 + ' x ' + time2,
              time1 + ' x ' + time4,
              time3 + ' x ' + time2]
  resultados = [jogo1[6],jogo2[6],jogo3[6],jogo4[6],jogo5[6],jogo6[6]]
  placares = [jogo1[-1],jogo2[-1],jogo3[-1],jogo4[-1],jogo5[-1],jogo6[-1]]

  jogos = pd.DataFrame([partidas,placares,resultados]).T 
  jogos.columns = ['Partida','Placar','Resultado'] 

  return [tab,jogos]

grupo = Jogos_grupo(selecoes,"G")
grupo[0]

,Pontos,Saldo de Gols,Gols Pró,Posição
Brasil,7,5,7,1
Suíça,5,2,4,2
Camarões,2,-4,3,3
Sérvia,1,-3,1,4


## Fase eliminátoria

In [13]:
def jogo_mata(selecao1,selecao2):
  jogo = Jogo(selecao1,selecao2)
  resultado = jogo[6]
  if resultado == 'V':
    return selecao1
  elif resultado == 'D':
    return selecao2
  else:
    return random.sample([selecao1,selecao2],1)[0]

In [14]:
def Simula_copa(selecoes):
  colunas = ['1st','2sd','3rd','4th','Oitavas','Quartas','Semis','Final','Campeão']
  n = selecoes.shape[0]
  m = len(colunas)
  aux = np.array(np.zeros(n*m).reshape(n,m))
  info = pd.DataFrame(aux,columns=colunas,index= selecoes.index)
  info = info.astype(int)
  info

  top16 = []
  for i in ('A','B','C','D','E','F','G','H'):
    t = Jogos_grupo(selecoes,i)[0]
    top16 += t.index[:2].tolist()
    anomes = t.index.tolist()
    info.at[anomes[0],'1st'] = 1
    info.at[anomes[0],'2sd'] = 1
    info.at[anomes[0],'3rd'] = 1
    info.at[anomes[0],'4th'] = 1

  qf1 = jogo_mata(top16[0],top16[3])
  qf2 = jogo_mata(top16[2],top16[1])
  qf3 = jogo_mata(top16[4],top16[7])
  qf4 = jogo_mata(top16[6],top16[5])
  qf5 = jogo_mata(top16[8],top16[11])
  qf6 = jogo_mata(top16[10],top16[9])
  qf7 = jogo_mata(top16[12],top16[15])
  qf8 = jogo_mata(top16[14],top16[13])

  top8 = [qf1,qf2,qf3,qf4,qf5,qf6,qf7,qf8]

  sf1 = jogo_mata(qf1,qf3)
  sf2 = jogo_mata(qf2,qf4)
  sf3 = jogo_mata(qf5,qf7)
  sf4 = jogo_mata(qf6,qf8)

  top4 = [sf1,sf2,sf3,sf4]

  fin1 = jogo_mata(sf1,sf3)
  fin2 = jogo_mata(sf2,sf4)

  top2 = [fin1,fin2]
  campeao = jogo_mata(fin1,fin2)

  info.at[top16,'Oitavas'] = 1
  info.at[top8,'Quartas'] = 1
  info.at[top4,'Semis'] = 1
  info.at[top2,'Final'] = 1
  info.at[campeao,'Campeão'] = 1

  return info

In [15]:
# simulando a competição 100 vezes 
def Simulacao(selecoes,n=100):
  print('IA.... Iniciando a simulação')
  info = Simula_copa(selecoes)
  for i in range(n-1):
    info += Simula_copa(selecoes)

  print('IA.... Simulação Finalizada')
  return info.sort_values(by='Campeão',ascending=False)/n


  

In [19]:
sim = Simulacao(selecoes,1000)
sim.to_excel('saidasimulaçãocopamundo2022.xlsx')
sim

IA.... Iniciando a simulação
IA.... Simulação Finalizada


,1st,2sd,3rd,4th,Oitavas,Quartas,Semis,Final,Campeão
Seleção,,,,,,,,,
Bélgica,0.592,0.592,0.592,0.592,0.844,0.558,0.343,0.210,0.133
Brasil,0.571,0.571,0.571,0.571,0.852,0.557,0.334,0.199,0.113
Argentina,0.522,0.522,0.522,0.522,0.821,0.515,0.331,0.191,0.096
França,0.532,0.532,0.532,0.532,0.809,0.449,0.265,0.156,0.091
Inglaterra,0.463,0.463,0.463,0.463,0.722,0.459,0.252,0.128,0.078
Espanha,0.428,0.428,0.428,0.428,0.719,0.381,0.210,0.128,0.065
Portugal,0.475,0.475,0.475,0.475,0.807,0.417,0.207,0.113,0.062
Holanda,0.523,0.523,0.523,0.523,0.828,0.452,0.232,0.095,0.051
Dinamarca,0.321,0.321,0.321,0.321,0.657,0.348,0.199,0.097,0.049


In [18]:
etapas = pd.DataFrame()
etapas['Cair na 1° Fase'] = 1 - sim['Oitavas']
etapas['Cair nas Oitavas'] = sim['Oitavas'] - sim['Quartas']
etapas['Cair nas Quartas'] = sim['Quartas'] - sim['Semis']
etapas['Cair nas Semis'] = sim['Semis'] - sim['Final']
etapas['Cair na Final'] = sim['Final'] - sim['Campeão']
etapas['Ganhar a Final'] = sim['Semis'] 
etapas.to_excel('saidaprobabilidadesPorEtapas.xlsx')
etapas 



,Cair na 1° Fase,Cair nas Oitavas,Cair nas Quartas,Cair nas Semis,Cair na Final,Ganhar a Final
Seleção,,,,,,
Brasil,0.15,0.26,0.22,0.10,0.09,0.37
França,0.21,0.35,0.09,0.18,0.06,0.35
Bélgica,0.21,0.30,0.22,0.12,0.06,0.27
Inglaterra,0.28,0.21,0.25,0.13,0.05,0.26
Espanha,0.23,0.34,0.24,0.09,0.03,0.19
Argentina,0.14,0.32,0.27,0.16,0.04,0.27
México,0.27,0.37,0.11,0.10,0.08,0.25
Dinamarca,0.33,0.35,0.10,0.09,0.06,0.22
Suíça,0.49,0.22,0.09,0.08,0.08,0.20
